In [4]:
import numpy
import visa

#numpy.frombuffer?

In [5]:
#usb = rigol.list_device()
#print(usb)
#adr = usb[0]

#adr = u'TCPIP::10.5.78.184::INSTR'
adr = u'USB0::0x1AB1::0x04CE::DS1ZA170300751::INSTR'

In [6]:
class ResourceManager(object):
    def __init__(self, **kwargs):
        #self.rm = visa.ResourceManager(**kwargs)
        #self.rm = visa.ResourceManager('@py',**kwargs)
        self.rm = visa.ResourceManager('@ni',**kwargs)

    def __enter__(self):
        return(self.rm)

    def __exit__(self, exc_type, exc, exc_tb):
        del(self.rm)


class Instrument(object):    
    def __init__(self, rm, resource_name, **kwargs):
        self.inst = rm.open_resource(resource_name, **kwargs)
    def __enter__(self):
        return(self.inst)

    def __exit__(self, exc_type, exc, exc_tb):
        self.inst.close()

In [7]:
#usb = list(filter(lambda x: 'USB' in x, instruments))
#adr = usb[0]

#adr = "TCPIP::10.5.78.184::INSTR"

#scope = rm.open_resource(usb[0], 
#                         timeout=20, 
#                         chunk_size=1024000, 
#                         write_termination=u'\r', #u'\n\x00'
#                         read_termination=u'\r',  #u'\n\x00'
#                         encoding = 'utf8' 
#                         )
#scope.data_bits=5
#scope.StopBits=1
#scope.SerialTermination.last_bit = scope.SerialTermination.last_bit

with ResourceManager() as rm:
    instruments = rm.list_resources()

    #with Instrument(rm, adr, encoding = 'utf8') as scope:
    with Instrument(rm, adr) as scope:
        print(scope.query(u'*IDN?\n'))
        scope.write(":STOP")

        timescale = float(scope.query(u':TIM:SCAL?\n'))
        timeoffset = float(scope.query(u':TIM:OFFS?\n'))
        voltscale = float(scope.query(u':CHAN1:SCAL?\n'))
        voltoffset = float(scope.query(u':CHAN1:OFFS?\n'))
        sample_rate = scope.query(u':ACQuire:SRATe?\n')

        scope.write(u':WAVeform:MODE RAW\n')
        scope.write(u':WAVeform:FORMat BYTE\n')
        #rawdata = scope.query(":WAV:DATA? CHAN1").encode('ascii')[10:]
        rawdata = scope.query_binary_values(u':WAV:DATA? CHAN1\n',
                                            datatype='B',
                                            container = numpy.array,
                                            is_big_endian  =False)
        #print([chr(x) for x in rawdata[0:10]])
        #print([255-x for x in rawdata[0:10]])
        #rawdata = scope.query_binary_values(":WAV:DATA? CHAN1")
        #print([hex(x) for x in rawdata])
        #print(bin(rawdata))
        #rawdata = scope.read_raw()
        #print(rawdata)
        rawdata = rawdata[10:-1]
            
        scope.write(u':RUN\n')
        #scope.close()

data = numpy.frombuffer(rawdata[:-1], 'B')
#print(data)
data =  255 - data
#print(data)
data = (data - 130.0 - voltoffset/voltscale*25) / 25 * voltscale
#print(data)

time = numpy.linspace(timeoffset - 6 * timescale, timeoffset + 6 * timescale, num=len(data))

RIGOL TECHNOLOGIES,DS1054Z,DS1ZA170300751,00.04.02.SP4

